In [1]:
from glob import glob
from os import path
import pandas as pd


# Import data

List all files to explore

In [2]:
glob(path.join('DB/institutes','*.csv'))

['DB/institutes\\institutes_academiainstitute.csv',
 'DB/institutes\\institutes_basicservicesupplierinstitute.csv',
 'DB/institutes\\institutes_financialinstitute.csv',
 'DB/institutes\\institutes_institute.csv',
 'DB/institutes\\institutes_investorinstitute.csv',
 'DB/institutes\\institutes_technicalassistanceinstitute.csv']

Ingest all files in the selected directory

In [3]:
file_names = glob(path.join('DB/institutes','*.csv'))
dfs = [pd.read_csv(fn) for fn in file_names]
institutes_df = pd.concat(dfs)

#print(institutes_df.info())

In [4]:
#print(list(institutes_df.columns.values))

We will export the DataFrame to have a look using Excel. 

In [5]:
#institutes_df.to_csv('institutes_df.csv')

According to this dataset, some fielda are not relevant to find the nearest neighbors (matches).
This fields are:
- id: this field is an identification number.
- website_url: this field is a webpage.
- date_of_establishment: this field is a date of establishment **might not be relevant. Or is it?**
- address_id: this field looks like a running number. **Is it relevant?**
- mission: this field is an open text. We could apply NLP to identify topics. **Maybe in a version 2 of the tool.**
- vision: this field is an open text. We could apply NLP to identify topics. **Maybe in a version 2 of the tool.**
- avatar: this field is an image.

# Build DataFrame with relevant features

In [6]:
institutes_df_short = institutes_df.drop(['website_url','date_of_establishment','address_id','mission','vision','avatar'], axis=1)

Replace all blancs (white spaces) with an empty string [ ]

In [7]:
institutes_df_short = institutes_df_short.replace(r'^\s*$', np.nan, regex=True)
#print(institutes_df_short.head())

Replace all empty strings {} with NaN

Let's check in Excel...

In [8]:
#institutes_df_short.to_csv('institutes_df_short.csv')

In [9]:
#print(institutes_df_short)

# Split DataFrame with features of type "object"

Let's separate the columns containing dtype == object to prepare the list to a standard form. This step will not generate yet an object recognized as a string.
Now, clean the strings from inconsistant characters such as brackets, quotatios, curly brackets, etc.

In [10]:
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].replace(np.nan, "[]",regex=True)
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace('"', "",regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace('{', "[",regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace('}', "]",regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace(', ', '","',regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace(',', '","',regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace('[', '"',regex=True))
institutes_df_short = institutes_df_short.loc[:,institutes_df_short.dtypes==object].apply(lambda s:s.str.replace(']', '"',regex=True))
#institutes_df_short.head(20)

After this first step, our lists are finally recognized as such by Pandas. Still, we can not use the standard functions, because they are not made for list applications.

In [11]:
dfs = []
for index, column in enumerate(institutes_df_short):
    col_names = np.arange(0, len(pd.DataFrame(institutes_df_short.iloc[:,index].apply(lambda x: x[0:].split(',')).apply(pd.Series)).columns)).astype(str)
    df = institutes_df_short[column].apply(lambda x: x[0:].split(',')).apply(pd.Series).replace('""', np.nan,regex=True).reset_index(drop=True)
    df.columns = ["{}{}".format(column, i) for i in col_names]
    dfs.append(df)
#dfs

In [12]:
# to merge the array of DataFrames into a single Dataframe
from functools import reduce
institutes_df_short_o = reduce(lambda x,y: pd.merge(x,y, left_index=True, right_index=True), dfs)
#institutes_df_short_o

# Split DataFrame with features not of type "object"

Now, let's separate the columns containing dtype != object to include to the dataframe with the lists created in the step before.

In [13]:
institutes_df_short_no = institutes_df.loc[:,institutes_df.dtypes!=object]
#institutes_df_short_no.info()

# One hot encoding 

To devide the categorical data into dummy variables we need to convert only the variables of type 'object'

In [14]:
# Make a list of column names for variables of the type object
object_variables = list(institutes_df_short_o.columns)
#print(object_variables)

In [15]:
Cat_X = pd.get_dummies(institutes_df_short_o, columns=object_variables)

Let's check in Excel...

In [16]:
#Cat_X.to_csv('Cat_X.csv')

# Mergeing categorical and non-categorical DataFrames

In [17]:
not_object_variables = list(institutes_df_short_no.columns)
#print(not_object_variables)

In [18]:
# Saving the column id in a DataFrame
id_X = institutes_df_short_no['id']

In [19]:
# Redefining the DataFrame with non categorical values removing the column id
nonCat_X = institutes_df_short_no.drop(['id', 'mixmarket_profile', 'current_financial_needs','object_id', 'address_id', 'content_type_id'], axis=1)
#nonCat_X.to_csv('nonCat_X.csv')

In [20]:
X = reduce(lambda x,y: pd.merge(x,y, left_index=True, right_index=True), [Cat_X,nonCat_X]).replace(np.nan, 0)
#X.to_csv('X.csv')
#X

# Nearest neighbors KNN

In [21]:
# Finding the K nearest neighbors, where number of neighbors is selected in n_neighbors

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(X)
distances, indices = nbrs.kneighbors(X)

In [22]:
#indices.shape

In [23]:
np.savetxt("indices of closest neighbors.csv", indices, delimiter=",")
#print(indices) # indices of closest neighbors